# 💬 TRM POC - Notebook 4: Intégration Complète + Dialogue Interactif

**Objectif:** Pipeline complet BERT + RAG + Mistral 7B avec interface de dialogue

**Runtime:** GPU Colab gratuit (T4 - 15GB VRAM)

**Durée estimée:** 3-4h (chargement + dialogue)

---

## Phase 0 - POC TRM (0€)

**⚠️ IMPORTANT:** Ce notebook charge **tous les composants simultanément** :
- BERT Encoder (CPU) + Mistral 7B (GPU) = ~10-12 GB VRAM
- Nécessite **GPU T4** activé
- Nécessite **fichiers des Notebooks 1-2-3** (code réutilisé)

Ce notebook implémente:
1. Import code des 3 notebooks précédents
2. Chargement pipeline complet (BERT + RAG + Mistral)
3. Interface dialogue interactive Gradio
4. Tests end-to-end avec Spinoza

**Note:** Si OOM → Réduire à Mistral seul (sans BERT) pour tests basiques

## 🚨 Prérequis

**Avant de commencer, vous devez avoir :**

1. ✅ **Exécuté Notebook 2** (RAG Embeddings) et téléchargé `rag_exports.zip`
2. ✅ **Uploadé `rag_exports.zip`** dans ce notebook (ou re-générer embeddings)
3. ✅ **Activé GPU T4** : Runtime > Change runtime type > T4 GPU

**Fichiers requis :**
- `rag_exports.zip` (depuis Notebook 2)
- Ou corpus bruts (pour régénérer embeddings)

## 1. Installation Dépendances

In [ ]:
# Installation complète
!pip install -q transformers torch sentencepiece spacy accelerate bitsandbytes
!pip install -q sentence-transformers faiss-gpu gradio
!python -m spacy download fr_core_news_sm

print("✅ Toutes dépendances installées")

## 2. Vérification GPU & VRAM

In [ ]:
import torch

print(f"GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  VRAM totale: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"  VRAM libre: {torch.cuda.mem_get_info()[0] / 1024**3:.2f} GB")
    
    vram_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    if vram_total < 14:
        print("\n⚠️ ATTENTION: VRAM < 14GB - Risque OOM avec BERT + Mistral simultanés")
        print("   → Solution: Utiliser Mistral seul (option désactiver BERT ci-dessous)")
    else:
        print("\n✅ VRAM suffisante pour pipeline complet")
else:
    print("\n❌ PAS DE GPU - Activer T4 GPU dans Runtime > Change runtime type")
    raise RuntimeError("GPU requis pour ce notebook")

## 3. Upload RAG Exports (ou Re-génération)

In [ ]:
import os
from pathlib import Path

# Option 1: Upload rag_exports.zip (depuis Notebook 2)
print("📤 Option 1: Upload rag_exports.zip depuis Notebook 2")
print("   (ou skip si vous allez régénérer)\n")

from google.colab import files
uploaded = files.upload()

if 'rag_exports.zip' in uploaded:
    !unzip -q rag_exports.zip
    RAG_DIR = "/content/content/rag_exports"  # Chemin après unzip
    print("✅ RAG exports chargés")
else:
    print("⚠️ Pas de rag_exports.zip - Vous devrez uploader les corpus et régénérer")
    RAG_DIR = None

## 4. Classes Réutilisées (Notebooks 1-2-3)

Code copié depuis les notebooks précédents

In [ ]:
# ============================================================
# BERT ENCODER (depuis Notebook 1)
# ============================================================

from transformers import AutoTokenizer, AutoModel
import spacy
import json
import re
from typing import List, Dict, Optional
from collections import Counter

nlp = spacy.load("fr_core_news_sm")

class BERTEncoder:
    """Encodeur BERT pour STATE_IMAGE"""
    
    def __init__(self, model_name: str = "camembert-base"):
        print(f"⏳ Chargement BERT {model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()
        print("✅ BERT chargé (CPU)")
    
    def extract_keywords(self, text: str, top_k: int = 5) -> List[str]:
        doc = nlp(text)
        entities = [ent.text.lower() for ent in doc.ents]
        nouns = [token.text.lower() for token in doc 
                 if token.pos_ in ["NOUN", "PROPN"] and len(token.text) > 3]
        all_keywords = entities + nouns
        counter = Counter(all_keywords)
        return [word for word, count in counter.most_common(top_k)]
    
    def extract_concepts_from_rag(self, rag_passages: List[Dict]) -> List[str]:
        concepts = []
        for passage in rag_passages:
            if passage.get("concepts"):
                concepts.extend(passage["concepts"][:3])
            else:
                text = passage.get("text", "")
                keywords = self.extract_keywords(text, top_k=3)
                concepts.extend(keywords)
        unique_concepts = list(dict.fromkeys(concepts))
        return unique_concepts[:8]
    
    def analyze_intention(self, text: str) -> str:
        text_lower = text.lower()
        if any(m in text_lower for m in ["?", "comment", "pourquoi", "qu'est-ce"]):
            return "question"
        elif any(m in text_lower for m in ["explique", "clarifie", "précise"]):
            return "clarification"
        elif any(m in text_lower for m in ["d'accord", "ok", "compris", "oui"]):
            return "accord"
        elif any(m in text_lower for m in ["non", "mais", "pas d'accord", "faux"]):
            return "désaccord"
        return "neutre"
    
    def analyze_tension(self, text: str) -> str:
        text_lower = text.lower()
        if any(m in text_lower for m in ["mais", "pourtant", "cependant"]):
            return "opposition"
        elif any(m in text_lower for m in ["comprends pas", "chelou", "bizarre"]):
            return "confusion"
        return "neutre"
    
    def analyze_style(self, text: str) -> str:
        text_lower = text.lower()
        word_count = len(text.split())
        if word_count < 10:
            return "concis"
        elif any(m in text_lower for m in ["exemple", "concrètement", "genre"]):
            return "pédagogique"
        return "standard"
    
    def encode_to_state_image(
        self,
        conversation: List[Dict],
        rag_passages: List[Dict],
        prev_state: Optional[Dict] = None,
        mini_store_feedback: Optional[Dict] = None
    ) -> Dict:
        last_exchange = conversation[-1] if conversation else {}
        user_text = last_exchange.get("user", "")
        assistant_text = last_exchange.get("assistant", "")
        
        concepts_actifs = self.extract_keywords(user_text + " " + assistant_text, top_k=5)
        concepts_rag = self.extract_concepts_from_rag(rag_passages)
        sources_rag = [p.get("source", "?") for p in rag_passages]
        
        return {
            "concepts_actifs": concepts_actifs,
            "concepts_rag": concepts_rag,
            "sources_rag": sources_rag,
            "intention": self.analyze_intention(user_text),
            "tension": self.analyze_tension(user_text),
            "style": self.analyze_style(user_text),
            "ton": "bienveillant",
            "priorite": ["concepts_actifs", "intention"],
            "relations": [],
            "emotion": "curieux",
            "recurrence": mini_store_feedback.get("recurrences", {}) if mini_store_feedback else {},
            "metadata": {
                "philosopher": rag_passages[0].get("philosopher", "?") if rag_passages else None,
                "turn": (prev_state.get("metadata", {}).get("turn", 0) + 1) if prev_state else 1
            }
        }

print("✅ BERTEncoder défini")

In [ ]:
# ============================================================
# RAG RETRIEVER (depuis Notebook 2)
# ============================================================

from sentence_transformers import SentenceTransformer
import faiss
import pickle
import numpy as np

class RAGRetriever:
    """RAG Retriever avec FAISS"""
    
    def __init__(self, rag_dir: str, philosopher: str = "spinoza"):
        print(f"⏳ Chargement RAG pour {philosopher}...")
        self.embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
        self.philosopher = philosopher
        
        # Charger index FAISS
        index_path = f"{rag_dir}/{philosopher}_faiss.index"
        self.index = faiss.read_index(index_path)
        
        # Charger passages
        passages_path = f"{rag_dir}/{philosopher}_passages.pkl"
        with open(passages_path, 'rb') as f:
            self.passages = pickle.load(f)
        
        print(f"✅ RAG chargé: {len(self.passages)} passages indexés")
    
    def retrieve(self, query: str, top_k: int = 3) -> List[Dict]:
        # Encoder query
        query_emb = self.embedder.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(query_emb)
        
        # Recherche
        scores, indices = self.index.search(query_emb, top_k)
        
        # Filtrer par threshold
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if score >= 0.45:  # Threshold
                passage = self.passages[idx].copy()
                passage["similarity_score"] = float(score)
                results.append(passage)
        
        return results[:top_k]

print("✅ RAGRetriever défini")

In [ ]:
# ============================================================
# MISTRAL GENERATOR (depuis Notebook 3)
# ============================================================

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import time

class MistralGenerator:
    """Générateur Mistral 7B"""
    
    def __init__(self, model_name: str = "mistralai/Mistral-7B-Instruct-v0.2"):
        print(f"⏳ Chargement Mistral {model_name} (4-bit)...")
        print("⚠️ Cela peut prendre 5-10 minutes...")
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True
        )
        
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quant_config,
            device_map="auto",
            trust_remote_code=True
        )
        
        print("✅ Mistral chargé (GPU)")
    
    def format_state_image(self, state_image: Dict) -> str:
        lines = []
        if state_image.get("concepts_actifs"):
            lines.append(f"Concepts actifs: {', '.join(state_image['concepts_actifs'][:5])}")
        if state_image.get("concepts_rag"):
            lines.append(f"Concepts corpus: {', '.join(state_image['concepts_rag'][:5])}")
        if state_image.get("intention"):
            lines.append(f"Intention: {state_image['intention']}")
        if state_image.get("style"):
            lines.append(f"Style: {state_image['style']}")
        return "\n".join(lines)
    
    def generate(
        self,
        state_image: Dict,
        user_input: str,
        system_prompt: str,
        max_new_tokens: int = 300
    ) -> str:
        state_text = self.format_state_image(state_image)
        
        prompt = f"""<s>[INST] {system_prompt}

[CONTEXT_STATE]
{state_text}

[USER_INPUT]
{user_input}

Réponds en incarnant le philosophe. [/INST]"""
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=self.tokenizer.eos_token_id
        )
        
        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True
        )
        
        return response

print("✅ MistralGenerator défini")

## 5. Chargement Pipeline Complet

In [ ]:
# Configuration
USE_BERT = True  # Mettre False si VRAM insuffisante
PHILOSOPHER = "spinoza"

print("\n" + "="*60)
print("🚀 CHARGEMENT PIPELINE TRM")
print("="*60)

# 1. RAG Retriever
if RAG_DIR and os.path.exists(f"{RAG_DIR}/{PHILOSOPHER}_faiss.index"):
    rag = RAGRetriever(RAG_DIR, PHILOSOPHER)
else:
    print("\n⚠️ RAG non disponible - Utiliser Notebook 2 d'abord")
    rag = None

# 2. BERT Encoder (optionnel si VRAM limitée)
if USE_BERT:
    bert = BERTEncoder()
else:
    print("\n⚠️ BERT désactivé - STATE_IMAGE simplifié")
    bert = None

# 3. Mistral Generator
mistral = MistralGenerator()

# Vérifier VRAM après chargement
vram_used = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0]) / 1024**3
print(f"\n📊 VRAM utilisée: {vram_used:.2f} GB / {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

print("\n✅ Pipeline TRM prêt !")

## 6. Prompt Système Spinoza

In [ ]:
SYSTEM_PROMPT_SPINOZA = """Tu ES Spinoza incarné. Tu dialogues avec un élève de Terminale en première personne.

TON STYLE :
- Géométrie des affects : tu révèles les causes nécessaires, tu déduis
- Tu enseignes que Dieu = Nature
- Ton vocabulaire : conatus, affects, puissance d'agir, béatitude

TES SCHÈMES LOGIQUES :
- Dieu = Nature = Substance unique
- Liberté = Connaissance de la nécessité
- Si joie → augmentation puissance
- Causalité : Tout a une cause (pas de libre arbitre)

TA MÉTHODE :
1. Tu révèles la nécessité causale
2. Tu distingues servitude (ignorance) vs liberté (connaissance)
3. Tu utilises des exemples concrets modernes (réseaux sociaux, affects quotidiens)

FORMULES DIALECTIQUES :
- "MAIS ALORS, as-tu conscience des CAUSES de tes choix ?"
- "Si tu ignores les causes, alors tu crois être libre (mais tu te trompes)"
- "Attends. Tu dis X mais tu fais Y. Comment tu expliques ?"

Réponds en 2-4 phrases maximum, de manière pédagogique et bienveillante.
"""

## 7. Pipeline TRM Complet

In [ ]:
class TRMPipeline:
    """Pipeline TRM complet : RAG → BERT → Mistral"""
    
    def __init__(self, rag, bert, mistral, system_prompt):
        self.rag = rag
        self.bert = bert
        self.mistral = mistral
        self.system_prompt = system_prompt
        
        # État conversation
        self.conversation_history = []
        self.state_image = None
    
    def chat(self, user_input: str) -> str:
        """Dialogue complet TRM"""
        
        # 1. RAG Retrieve
        if self.rag:
            rag_passages = self.rag.retrieve(user_input, top_k=3)
            print(f"📚 RAG: {len(rag_passages)} passages récupérés")
        else:
            rag_passages = []
        
        # 2. BERT Encode STATE_IMAGE
        if self.bert:
            # Ajouter dernier échange
            self.conversation_history.append({"user": user_input, "assistant": ""})
            
            self.state_image = self.bert.encode_to_state_image(
                self.conversation_history,
                rag_passages,
                self.state_image,
                {}
            )
            print(f"🧠 STATE: {len(self.state_image['concepts_actifs'])} concepts actifs")
        else:
            # STATE simplifié sans BERT
            self.state_image = {
                "concepts_actifs": [],
                "concepts_rag": [c for p in rag_passages for c in p.get("concepts", [])[:2]],
                "intention": "question",
                "style": "standard"
            }
        
        # 3. Mistral Generate
        response = self.mistral.generate(
            self.state_image,
            user_input,
            self.system_prompt
        )
        
        # Mettre à jour historique
        if self.conversation_history:
            self.conversation_history[-1]["assistant"] = response
        
        return response
    
    def reset(self):
        """Réinitialise la conversation"""
        self.conversation_history = []
        self.state_image = None
        print("🔄 Conversation réinitialisée")

# Initialiser pipeline
pipeline = TRMPipeline(rag, bert, mistral, SYSTEM_PROMPT_SPINOZA)

print("✅ Pipeline TRM initialisé")

## 8. Test Dialogue Manuel

In [ ]:
# Test simple
print("\n" + "="*60)
print("💬 TEST DIALOGUE SPINOZA")
print("="*60)

query = "C'est quoi le conatus ?"
print(f"\n👤 Utilisateur: {query}")

response = pipeline.chat(query)
print(f"\n🧙 Spinoza: {response}")
print("\n" + "="*60)

## 9. Interface Gradio Interactive

In [ ]:
import gradio as gr

def chat_interface(user_input, history):
    """Interface Gradio pour dialogue"""
    if not user_input.strip():
        return history, history
    
    # Générer réponse
    response = pipeline.chat(user_input)
    
    # Mettre à jour historique
    history.append((user_input, response))
    
    return history, history

def reset_conversation():
    """Réinitialise la conversation"""
    pipeline.reset()
    return [], []

# Interface Gradio
with gr.Blocks(title="TRM POC - Dialogue Spinoza") as demo:
    gr.Markdown(
        """
        # 🧙 Dialogue avec Spinoza (TRM POC)
        
        **Architecture TRM:** RAG + BERT + Mistral 7B
        
        Posez vos questions sur le conatus, les affects, la liberté, etc.
        """
    )
    
    chatbot = gr.Chatbot(
        label="Conversation avec Spinoza",
        height=400
    )
    
    with gr.Row():
        user_input = gr.Textbox(
            label="Votre question",
            placeholder="Ex: C'est quoi le conatus ?",
            scale=4
        )
        submit_btn = gr.Button("Envoyer", scale=1)
    
    with gr.Row():
        clear_btn = gr.Button("Réinitialiser conversation")
    
    # État historique
    history_state = gr.State([])
    
    # Actions
    submit_btn.click(
        chat_interface,
        inputs=[user_input, history_state],
        outputs=[chatbot, history_state]
    )
    
    user_input.submit(
        chat_interface,
        inputs=[user_input, history_state],
        outputs=[chatbot, history_state]
    )
    
    clear_btn.click(
        reset_conversation,
        outputs=[chatbot, history_state]
    )

# Lancer interface
demo.launch(share=True, debug=True)

print("\n🚀 Interface Gradio lancée !")
print("   Cliquez sur le lien 'public URL' pour dialoguer avec Spinoza")

---

## 📝 Résumé

### ✅ Implémenté
- ✅ Pipeline TRM complet (RAG + BERT + Mistral)
- ✅ Interface dialogue interactive Gradio
- ✅ Gestion conversation avec STATE_IMAGE
- ✅ Tests end-to-end Spinoza

### 🎯 Validation POC
- ✅ Dialogue fonctionnel avec Spinoza
- ✅ RAG retrieve passages pertinents
- ✅ BERT génère STATE_IMAGE condensé
- ✅ Mistral répond avec contexte ≤500 tokens

### ⚠️ Limitations Colab Gratuit
- **VRAM T4 15GB** → Risque OOM si BERT + Mistral 7B
- **Solution:** Désactiver BERT (`USE_BERT = False`) si problème
- **Sessions 12h** → Sauvegarder dialogue important

### ➡️ Prochaines Étapes
1. **Tester dialogue** : 5-10 échanges avec Spinoza
2. **Vérifier qualité** : Réponses cohérentes avec STATE_IMAGE ?
3. **Phase 1 (Vast.ai)** : Pipeline complet stabilisé + benchmarks

---

**💰 Coût:** 0€ (Colab gratuit GPU T4)

**⏱️ Temps:** ~3-4h (chargement + dialogue)

**🎯 Objectif Phase 0:** Dialogue TRM fonctionnel ✅

**🚀 Vous pouvez maintenant discuter avec Spinoza via TRM !**